# Proyecto individual: Sistema de recomendación de películas
​
Este proyecto constará de dos fases: `Ingenieria de datos`, `Modelamiento y evaluación con machine learning`.

### 1. Ingeniería de datos
* Esto incluye la limpieza y transformación de los datos, abordando problemas como:
    * valores faltantes, 
    * datos duplicados y variables irrelevantes,
    * valores anidados,
    * formateo de columnas,
    * nubes de palabras para ver las más frecuentes.
    * a fin de mejorar la calidad del dataset para el modelado.
    * análisis univariado.
    * análisis bivariado y multivariado.
​
### 2. Modelamiento y evaluación con machine learning
* Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar (**con un algoritmo de coseno de similitud, por ejemplo**) las películas por ... para encontrar una lista de 5 películas similares 

### 1. Transformaciones

Se realizará una transformación de datos para que corra los primeros 4 endpoints:
* Las variables a considerar serán:
    * release_date (year, month, weekday)
    * popularity
    * vote_average
    * vote_count

#### 1.1. Importación de librerías

In [14]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime

# Mostrar figuras de matplotlib en el entorno de Jupyter Notebook
%matplotlib inline

#### 1.2. Carga y visualización los datos.

In [15]:
df_movies = pd.read_csv('datasets/movies.csv')
df_movies.head(2)

C:\Users\Josue\AppData\Local\Temp\ipykernel_10276\3549431830.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('datasets/movies.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


#### 1.3. Eliminación de filas

1.3.1. Primero se van a eliminar las filas donde haya valores nulos en las columnas status (no fueron lanzadas todavía) y release_date (no tienen fecha y no sirve para los endpoints).

In [16]:
# Se filtran las filas con valores notna en las columnas 'status', 'release_date' 
df_movies_1 = df_movies[df_movies['status'].notna()] 
df_movies_1 = df_movies[df_movies['release_date'].notna()] 

1.3.2. El propósito es reducir el dataset y enfocar los datos, para ello se elige trabajar solo con las películas en inglés, las cuales representan el 70 % del total, dado que el cine en inglés es el más visualizado.

In [17]:
total_rows = len(df_movies)

count_en = (df_movies['original_language'] == 'en').sum()

prop_en = int(count_en / total_rows * 100)

print(f"La proporción del idioma inglés es del: {prop_en} %")

La proporción del idioma inglés es del: 70 %


In [18]:
# Se eliminan las películas que su idioma original no es en inglés (habria que analizar que porcentaje representan)
df_movies_1 = df_movies[df_movies['original_language'] == 'en']

1.3.3. Seguidamente, se elige trabajar con las películas mayores al año 1990, las cuales representan más del 75 % de los datos.

Este punto se resolverá luego de crear la columna release_year

#### 1.4. Eliminación de columnas 

In [19]:
df_movies_2 = df_movies_1.drop(['adult', 'belongs_to_collection', 'status', 'genres', 'poster_path', 'homepage', 'imdb_id', 'overview', 'production_companies', 'production_countries', 'original_language', 'runtime', 'spoken_languages', 'tagline', 'original_title', 'video', 'budget', 'revenue'], axis=1)


In [20]:
df_movies_2.head()

,id,popularity,release_date,title,vote_average,vote_count
0,862,21.946943,1995-10-30,Toy Story,7.7,5415.0
1,8844,17.015539,1995-12-15,Jumanji,6.9,2413.0
2,15602,11.7129,1995-12-22,Grumpier Old Men,6.5,92.0
3,31357,3.859495,1995-12-22,Waiting to Exhale,6.1,34.0
4,11862,8.387519,1995-02-10,Father of the Bride Part II,5.7,173.0


#### 1.5. Distinción entre day, month y year en la columna release_date

In [21]:
# Convertir la columna 'date' a datetime
df_movies_2['release_date'] = pd.to_datetime(df_movies_2['release_date'], errors='coerce')

# Verificar el formato de la columna 'date' después de la conversión
print(df_movies_2['release_date'].head())

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]


In [22]:
# Crear nuevas columnas para día, mes y año
df_movies_2['day'] = df_movies_2['release_date'].dt.day
df_movies_2['month'] = df_movies_2['release_date'].dt.month
df_movies_2['release_year'] = df_movies_2['release_date'].dt.year

In [23]:
df_movies_2.head(2)

,id,popularity,release_date,title,vote_average,vote_count,day,month,release_year
0,862,21.946943,1995-10-30,Toy Story,7.7,5415.0,30.0,10.0,1995.0
1,8844,17.015539,1995-12-15,Jumanji,6.9,2413.0,15.0,12.0,1995.0


In [24]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32269 entries, 0 to 45465
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            32269 non-null  object        
 1   popularity    32267 non-null  object        
 2   release_date  32202 non-null  datetime64[ns]
 3   title         32267 non-null  object        
 4   vote_average  32267 non-null  float64       
 5   vote_count    32267 non-null  float64       
 6   day           32202 non-null  float64       
 7   month         32202 non-null  float64       
 8   release_year  32202 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 2.5+ MB


In [25]:
df_movies_2.describe()

,release_date,vote_average,vote_count,day,month,release_year
count,32202,32267.000000,32267.00000,32202.000000,32202.000000,32202.000000
mean,1991-08-16 23:18:38.166573440,5.491171,141.56643,14.116111,6.448730,1991.135613
min,1878-06-14 00:00:00,0.000000,0.00000,1.000000,1.000000,1878.000000
25%,1978-05-03 12:00:00,5.000000,3.00000,6.000000,3.000000,1978.000000
50%,2000-12-27 00:00:00,5.900000,10.00000,14.000000,7.000000,2000.000000
75%,2010-10-08 00:00:00,6.700000,43.00000,22.000000,10.000000,2010.000000
max,2020-12-16 00:00:00,10.000000,14075.00000,31.000000,12.000000,2020.000000
std,NaN,1.941068,574.58508,9.262318,3.598641,24.711462


#### 1.6. Filtrado de películas mayores a 1980

Las cuales representan cerca del 75 % de los datos según lo muestra el .describe

In [48]:
df_movies_2 = df_movies_2[df_movies_2['release_year'] >= 1980]

In [49]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23802 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            23802 non-null  object        
 1   popularity    23802 non-null  object        
 2   release_date  23802 non-null  datetime64[ns]
 3   title         23802 non-null  object        
 4   vote_average  23802 non-null  float64       
 5   vote_count    23802 non-null  float64       
 6   day           23802 non-null  float64       
 7   month         23802 non-null  float64       
 8   release_year  23802 non-null  float64       
 9   weekday       23802 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 2.0+ MB


## 2. Pruebas de las funciones para los endpoints

### 2.1. Creación de la funcion para el endpoint1

In [28]:
month_mapping = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

In [29]:
def cantidad_filmaciones_mes(df, month: str):
    """Obtiene la cantidad de filmaciones según mes indicado
    Args:
        df: DataFrame de películas 
        month (str): nombre del mes en castellano
    Raises:
        HTTPException: en caso de que el mes indicado no sea válido o no esté escrito en castellano o sea nulo
    Returns:
        _type_: entero o integer
    """
    # Convertir el nombre del mes a su número correspondiente
    month_number = month_mapping.get(month.lower())
    if month_number is None:
        raise ValueError(f"Mes '{month}' no es válido.")
    # Filtrar las filas que corresponden al mes especificado
    filtered_df = df[df['month'] == month_number]
    # Contar los IDs en las filas filtradas
    count_ids = filtered_df['id'].count()
    return {"endpoint1": f"Fueron estrenadas {count_ids} peliculas en el mes de {month}"}

In [30]:
cantidad_filmaciones_mes(df=df_movies_2, month='enero')

{'endpoint1': 'Fueron estrenadas 3204 peliculas en el mes de enero'}

### 2.2. Creación de la funcion para el endpoint2

In [31]:
# Función para obtener el día de la semana
def obtener_dia_semana(fecha):
    dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    if pd.notnull(fecha):
        return dias_semana[int(fecha.day_of_week)]
    else:
        return 'fecha inválida'

# Aplicar la función a la columna 'release_date'
df_movies_2['weekday'] = df_movies_2['release_date'].apply(obtener_dia_semana)

# Se eliminan las filas con valor de celda 'fecha inválida'
df_movies_2 = df_movies_2[df_movies_2['weekday'] != 'fecha inválida']

In [32]:
df_movies_2.head(3)

,id,popularity,release_date,title,vote_average,vote_count,day,month,release_year,weekday
0,862,21.946943,1995-10-30,Toy Story,7.7,5415.0,30.0,10.0,1995.0,lunes
1,8844,17.015539,1995-12-15,Jumanji,6.9,2413.0,15.0,12.0,1995.0,viernes
2,15602,11.7129,1995-12-22,Grumpier Old Men,6.5,92.0,22.0,12.0,1995.0,viernes


In [33]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23802 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            23802 non-null  object        
 1   popularity    23802 non-null  object        
 2   release_date  23802 non-null  datetime64[ns]
 3   title         23802 non-null  object        
 4   vote_average  23802 non-null  float64       
 5   vote_count    23802 non-null  float64       
 6   day           23802 non-null  float64       
 7   month         23802 non-null  float64       
 8   release_year  23802 non-null  float64       
 9   weekday       23802 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 2.0+ MB


##### 2.2.1. Aclaración
Se agrega la columna weekday al dataset porque la memoria usada casi no se incrementa.

In [34]:
data_movies = df_movies_2.to_csv('dataset/data_movies.csv')

In [35]:
df = pd.read_csv('dataset/data_movies.csv')

In [36]:
from fastapi import HTTPException

def cantidad_filmaciones_dia(df, weekday: str):
    """Obtiene la cantidad de filmaciones según día de la semana indicado
    Args:
        df: DataFrame de películas 
        weekday (str): nombre del día de la semana en castellano
    Raises:
        HTTPException: en caso de que el día de la semana indicado no sea válido o no esté escrito en castellano o sea nulo
    Returns:
        _type_: entero o integer
    """
    # Lista de días válidos en castellano
    dias_validos = ['lunes', 'martes', 'miércoles',
                    'jueves', 'viernes', 'sábado', 'domingo']
    if weekday not in dias_validos:
        raise HTTPException(
            status_code=400, detail=f"El día '{weekday}' no es válido.")
    # Filtrar las filas que corresponden al dia de la semana especificado
    df_filter_day = df[df['weekday'] == weekday]
    # Contar los IDs en las filas filtradas
    count_ids = df_filter_day['id'].count()
    return {"endpoint2": f"Fueron estrenadas {count_ids} peliculas el dia {weekday}"}

In [37]:
lunes = cantidad_filmaciones_dia(df, 'domingo')
lunes

{'endpoint2': 'Fueron estrenadas 2103 peliculas el dia domingo'}

### 2.3. Creación de la funcion para el endpoint3

In [54]:
from fastapi import HTTPException

def score_titulo(df, titulo_de_la_filmacion: str): 
    """Obtiene el año de estreno y la popularidad de la película ingresada

    Args:
        df: DataFrame de películas
        titulo_de_la_filmacion (str): nombre de la película
    Raises:
        HTTPException: en caso de que la película no esté escrita de forma correcta o no esté ingresada en el dataset o sea inexistente.
    Returns:
        _type_: str
    """
    # Verificar si el título de la película está en el DataFrame
    if titulo_de_la_filmacion not in df['title'].values:
        raise HTTPException(
            status_code=400, detail=f"La película '{titulo_de_la_filmacion}' no ha sido estrenada o no está registrada en el dataset o es inexistente.")
    
    # Filtrar la fila que corresponde a la película especificada
    pelicula = df[df['title'] == titulo_de_la_filmacion]
    
    # Extrar año de estreno y popularidad
    release_year = int(pelicula['release_year'].values[0])
    popularity = round(pelicula['popularity'].values[0], 2)
    return {'endpoint3': f"La película {titulo_de_la_filmacion} fue estrenada en el año {release_year} con una popularidad de {popularity}"}

In [55]:
endpoint3 = score_titulo(df, 'Toy Story')
print(endpoint3)

{'endpoint3': 'La película Toy Story fue estrenada en el año 1995 con una popularidad de 21.95'}


### 2.4. Creación de la funcion para el endpoint4

In [52]:
from fastapi import HTTPException

def votos_titulo(df, titulo_de_la_filmacion: str): 
    """Obtiene el año de estreno, un total de valoraciones y un promedio según película ingresada

    Args:
        df: DataFrame de películas
        titulo_de_la_filmacion (str): nombre de la película
    Raises:
        HTTPException: en caso de que la película no esté escrita de forma correcta, no esté ingresada en el dataset o sea inexistente. Además, debe contar con al menos 2000 valoraciones.
    Returns:
        dict: Contiene el año de estreno, total de valoraciones y el promedio de valoraciones.
    """
    # Verificar si el título de la película está en el DataFrame
    if titulo_de_la_filmacion not in df['title'].values:
        raise HTTPException(
            status_code=400, detail=f"Puede ser que la película '{titulo_de_la_filmacion}': no haya sido estrenada, no esté registrada en el dataset, sea inexistente o no cuente con un mínimo de 2000 valoraciones.")
    
    # Filtrar la fila que corresponde a la película especificada
    pelicula = df[df['title'] == titulo_de_la_filmacion]
    
    # Extrar año de estreno y popularidad
    release_year = int(pelicula['release_year'].values[0])
    vote_count = int(pelicula['vote_count'].values[0])
    vote_average = round(pelicula['vote_average'].values[0], 1)
    
    if vote_count < 2000:
        return {'endpoint4': f"La película {titulo_de_la_filmacion} no cuenta con un mínimo de 2000 valoraciones y por tanto no se devuelve ningún valor."}
    
    return {'endpoint4': f"La película {titulo_de_la_filmacion} fue estrenada en el año {release_year}, cuenta con un total de {vote_count} valoraciones y un promedio de {vote_average} puntos."}

In [53]:
endpoint4 = votos_titulo(df, 'Toy Story')
print(endpoint4)

{'endpoint4': 'La película Toy Story fue estrenada en el año 1995, cuenta con un total de 5415 valoraciones y un promedio de 7.7 puntos.'}
